# Model Selection

Testing out different models and selecting what seems to perform the best.

## Setup

In [61]:
import csv
import json
import os
import pandas as pd
from pandas import DataFrame
from datetime import datetime

data_dir = os.path.abspath('data')
# today = datetime.today().strftime('%Y%m%d')
today = datetime.strptime('2022-09-27', '%Y-%m-%d').strftime('%Y%m%d')

user_df = pd.read_csv(os.path.join(data_dir, f'user-{today}-enriched.csv'))

print('user', user_df.shape[0], 'rows')

user 556 rows


In [62]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   score        556 non-null    int64  
 1   id           556 non-null    int64  
 2   format       556 non-null    object 
 3   season       556 non-null    object 
 4   season_year  556 non-null    float64
 5   episodes     556 non-null    float64
 6   genres       556 non-null    object 
 7   source       556 non-null    object 
 8   tags         556 non-null    object 
 9   studios      556 non-null    object 
dtypes: float64(2), int64(2), object(6)
memory usage: 43.6+ KB


In [63]:
user_df.describe()

,score,id,season_year,episodes
count,556.000000,556.000000,556.000000,556.000000
mean,6.969424,46938.041367,2013.079137,14.302158
std,1.528398,46407.064461,7.724830,48.325906
min,1.000000,1.000000,1984.000000,1.000000
25%,6.000000,10082.000000,2011.000000,1.000000
50%,7.000000,21054.500000,2015.000000,12.000000
75%,8.000000,100170.000000,2018.000000,13.000000
max,10.000000,142329.000000,2022.000000,1035.000000


## Encode Categorical Data

In [64]:
# TODO: Explode genres, tags, studios ???
# TODO: categorical encoding



def encode_col_array(df: DataFrame, col: str) -> DataFrame:
    df[col] = df[col].apply(json.loads)
    exploded = df.explode(col) # expand to multiple rows

    # one hot encode exploded rows
    one_hot = pd.concat([df, pd.get_dummies(exploded[col], prefix='genre', prefix_sep='_')], axis=1)
    return one_hot.groupby('id').max().reset_index()

user_df = encode_col_array(user_df, 'genres')

pd.set_option('display.max_rows', len(user_df))
print(user_df.head(1))

   id  score format  season  season_year  episodes  \
0   1      9     TV  SPRING       1998.0      26.0   

                               genres    source  \
0  [ACTION, ADVENTURE, DRAMA, SCI-FI]  ORIGINAL   

                                                tags      studios  ...  \
0  ["SPACE", "CRIME", "EPISODIC", "ENSEMBLE CAST"...  ["SUNRISE"]  ...   

   genre_MECHA  genre_MUSIC  genre_MYSTERY  genre_PSYCHOLOGICAL  \
0            0            0              0                    0   

   genre_ROMANCE  genre_SCI-FI  genre_SLICE OF LIFE  genre_SPORTS  \
0              0             1                    0             0   

   genre_SUPERNATURAL  genre_THRILLER  
0                   0               0  

[1 rows x 28 columns]
